# Filter snowlines manually by clicking through images and assessing visually

In [5]:
import matplotlib.pyplot as plt
import IPython

In [10]:
%matplotlib widget

arrays = [[[1,2,3], [1,2,3]], [[1,2,3], [2,3,4]]]

for i, array in enumerate(arrays):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(array[0], array[1])
    plt.show()
    my_input = input('Does this snowline look good?')

<IPython.core.display.Javascript object>

Does this snowline look good? no


<IPython.core.display.Javascript object>

Does this snowline look good? no


In [8]:
!conda list

# packages in environment at /Users/raineyaberle/opt/anaconda3/envs/snow-cover-mapping:
#
# Name                    Version                   Build  Channel
affine                    2.4.0              pyhd8ed1ab_0    conda-forge
aiohttp                   3.8.4           py311h5547dcb_0    conda-forge
aiosignal                 1.3.1              pyhd8ed1ab_0    conda-forge
anyio                     3.6.2              pyhd8ed1ab_0    conda-forge
aom                       3.5.0                hf0c8a7f_0    conda-forge
appnope                   0.1.3              pyhd8ed1ab_0    conda-forge
argon2-cffi               21.3.0             pyhd8ed1ab_0    conda-forge
argon2-cffi-bindings      21.2.0          py311h5547dcb_3    conda-forge
asttokens                 2.2.1              pyhd8ed1ab_0    conda-forge
async-timeout             4.0.2              pyhd8ed1ab_0    conda-forge
attrs                     23.1.0             pyh71513ae_1    conda-forge
babel                     2.12.1        